In [1]:
floods_url = "https://bhuvan-gp1.nrsc.gov.in/bhuvan/gwc/service/wms?LAYERS=flood%3Aas_2019_17_07_06&TRANSPARENT=TRUE&SERVICE=WMS&VERSION=1.1.1&REQUEST=GetMap&STYLES=&FORMAT=image%2Fpng&SRS=EPSG%3A4326&BBOX=90,22.5,95.625,28.125&WIDTH=256&HEIGHT=256"

In [2]:
river_url = "https://bhuvan-vec2.nrsc.gov.in/bhuvan/gwc/service/wms/?LAYERS=river%3Aas_riv17&TRANSPARENT=TRUE&SERVICE=WMS&VERSION=1.1.1&REQUEST=GetMap&STYLES=&FORMAT=image%2Fpng&SRS=EPSG%3A4326&BBOX=90,22.5,95.625,28.125&WIDTH=256&HEIGHT=256"

In [3]:
import urllib.request
from netCDF4 import Dataset
import numpy as np
from numpy import asarray
import PIL

import xarray as xr
import rioxarray as rio
from PIL import Image
PIL.Image.MAX_IMAGE_PIXELS = 933120000

In [13]:
delta = 0.0439453125  ## Resolution of the most granular tile on Bhuvan portal.
BBOX ='90,23.994140625,95.625,28.125' #FUll Assam extent.

In [14]:
#Dynamic flood tile URL from Bhuvan portal
floods_url =  "https://bhuvan-gp1.nrsc.gov.in/bhuvan/gwc/service/wms?LAYERS=flood%3Aas_"+date_string+"&TRANSPARENT=TRUE&SERVICE=WMS&VERSION=1.1.1&REQUEST=GetMap&STYLES=&FORMAT=image%2Fpng&SRS=EPSG%3A4326&BBOX="+BBOX+"&WIDTH=256&HEIGHT=256"

In [22]:
def get_image_from_tile(BBOX,date_string):
    '''
    Downloads the image from the WMS tile and returns the path of downloaded image.
    Input parameters:
    BBOX: Bounding box coordinates(ln_w, lt_s, ln_e, lt_n)
    date_string: "yyyy_dd_mm_hh" hh is optional

    '''
    floods_url =  "https://bhuvan-gp1.nrsc.gov.in/bhuvan/gwc/service/wms?LAYERS=flood%3Aas_"+date_string+"&TRANSPARENT=TRUE&SERVICE=WMS&VERSION=1.1.1&REQUEST=GetMap&STYLES=&FORMAT=image%2Fpng&SRS=EPSG%3A4326&BBOX="+BBOX+"&WIDTH=256&HEIGHT=256"
    #print(floods_url)
    urllib.request.urlretrieve(floods_url, r"Tiles/"+date_string+"xx"+BBOX+".image")    
    return None #r"Tiles/"+date_string+"xx"+BBOX+".image"

In [101]:

date_string = "2021_13_07_06"
delta = 0.0439453125

count=0
ln_w = 90 
lt_s = 23.994140625

#Scraping the tile longitudinally first and then latitudinally.

while lt_s <= 28.125:
    lt_n = lt_s+delta
    print(count)
        
    while ln_w <= 96:
        ln_e = ln_w+delta   
        BBOX="{},{},{},{}".format(ln_w,lt_s,ln_e,lt_n)
        ## Download the tile image
        get_image_from_tile(BBOX,date_string)
           
        ln_w = ln_e
        
    
    lt_s = lt_n
    ln_w = 90
    
## All tile images are downloaded -- these have to be stitched.

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94


In [23]:
## BEfore running the next cell -- find the lowest and highest lat lons.
import glob
extension = 'image'
        #os.chdir(path)
result = glob.glob('Tiles/*.{}'.format(extension))

lats = []
lons = []
for file in result:
    lats.append(float(file.split(',')[-1].split('.image')[0]))
    lons.append(float(file.split(',')[-2]))

In [24]:
len(result)

13015

In [25]:
def merge_images(file1, file2,horizontal):
    """Merge two images into one
    Input parameters
    file1: path to first image file
    file2: path to second image file
    horizontal: True if the images are to be stitched horizontally
    
    returns the merged Image object
    """
    if type(file1) ==PIL.Image.Image:
        image1 = file1
    else:
        image1 = Image.open(file1)
    image2 = Image.open(file2)

    (width1, height1) = image1.size
    (width2, height2) = image2.size

    if horizontal==True:
        result_width = width1 + width2
        result_height = max(height1, height2)
        result = Image.new('L', (result_width, result_height))
        result.paste(im=image1, box=(0, 0))
        result.paste(im=image2, box=(width1, 0))
    else:
        result_height = height1 + height2
        result_width = max(width1,width2)
        result = Image.new('L', (result_width, result_height))
        result.paste(im=image1, box=(0, 0))
        result.paste(im=image2, box=(0, height1))
    #result.save('result.image')
    return result

In [120]:
c = 0
count=1
#We will first stitch images vertically (latitudinally) and then stitch them horizontally(longitudinally)
#There are 95 images per a vertical stretch
while c+95<13015:
    print(count)
    vert_images = result[c:c+95]
    vert_images.reverse()
    merged_image = vert_images[0]
    for image in vert_images[1:]:
        merged_image = merge_images(merged_image, image,horizontal=False)
    c=c+95
    count=count+1
    merged_image.save(r'vert/'+str(count)+'.jpg')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136


In [132]:
#Stitch images horizontally
import glob
import natsort

extension = 'jpg'
vert_imgs = glob.glob('vert/*.{}'.format(extension))
vert_imgs = natsort.natsorted(vert_imgs,reverse=False)

merged_image = vert_imgs[0]
print(vert_imgs[0])
for image in vert_imgs[1:]:
    print(image)
    merged_image = merge_images(merged_image, image,horizontal=True)

vert\2.jpg
vert\3.jpg
vert\4.jpg
vert\5.jpg
vert\6.jpg
vert\7.jpg
vert\8.jpg
vert\9.jpg
vert\10.jpg
vert\11.jpg
vert\12.jpg
vert\13.jpg
vert\14.jpg
vert\15.jpg
vert\16.jpg
vert\17.jpg
vert\18.jpg
vert\19.jpg
vert\20.jpg
vert\21.jpg
vert\22.jpg
vert\23.jpg
vert\24.jpg
vert\25.jpg
vert\26.jpg
vert\27.jpg
vert\28.jpg
vert\29.jpg
vert\30.jpg
vert\31.jpg
vert\32.jpg
vert\33.jpg
vert\34.jpg
vert\35.jpg
vert\36.jpg
vert\37.jpg
vert\38.jpg
vert\39.jpg
vert\40.jpg
vert\41.jpg
vert\42.jpg
vert\43.jpg
vert\44.jpg
vert\45.jpg
vert\46.jpg
vert\47.jpg
vert\48.jpg
vert\49.jpg
vert\50.jpg
vert\51.jpg
vert\52.jpg
vert\53.jpg
vert\54.jpg
vert\55.jpg
vert\56.jpg
vert\57.jpg
vert\58.jpg
vert\59.jpg
vert\60.jpg
vert\61.jpg
vert\62.jpg
vert\63.jpg
vert\64.jpg
vert\65.jpg
vert\66.jpg
vert\67.jpg
vert\68.jpg
vert\69.jpg
vert\70.jpg
vert\71.jpg
vert\72.jpg
vert\73.jpg
vert\74.jpg
vert\75.jpg
vert\76.jpg
vert\77.jpg
vert\78.jpg
vert\79.jpg
vert\80.jpg
vert\81.jpg
vert\82.jpg
vert\83.jpg
vert\84.jpg
vert\85.jpg


In [133]:
merged_image.save('final.jpg')

In [26]:
def get_grayscale_array(image_path):
    '''
    Converts the RGBA image into grayscale and returns the array.
    Input parameters
    image_path: file path of the image.
    
    Returns the grayscale image and its path.
    '''
    image = Image.open(image_path)
    grayscale_image = image.convert('L')
    return grayscale_image, asarray(grayscale_image)

In [8]:
def create_nc_from_images(lt_s,lt_n,ln_w,ln_e,image_path,image_name):
    '''
    Creates NetCDF4 files from the grayscale image using the BBOX coordinates.
    
    Input parameters:
    BBOX coordindates (lt_s, lt_n, ln_w, ln_e)
    image_path: filepath of the image to be converted.
    image_name: output file name
    
    Returns the filepath of the NetCDF4 file created.
    '''
    image = Image.open(image_path)
    grayscale_array = np.asarray(image)
    
    lt_array = np.linspace(lt_n,lt_s,grayscale_array.shape[0])
    ln_array = np.linspace(ln_w,ln_e,grayscale_array.shape[1])
    
    my_file = Dataset(r"NCs/"+image_name+'.nc','w',format='NETCDF4')
    lat_dim = my_file.createDimension('lat',grayscale_array.shape[0])
    lon_dim = my_file.createDimension('lon',grayscale_array.shape[1])
    time_dim = my_file.createDimension('time',None)
    
    latitudes = my_file.createVariable("lat", 'f4', ('lat',))
    longitudes = my_file.createVariable("lon", 'f4', ('lon',))
    time = my_file.createVariable('time', np.float32, ('time',))
    
    new_nc_variable = my_file.createVariable("Inundation", np.float32, ('time','lat','lon'))
    latitudes[:] = lt_array
    longitudes[:] = ln_array
    new_nc_variable[0,:,:] = grayscale_array
    
    my_file.close()
    
    return r"NCs/"+image_name+'.nc'

In [27]:
def create_tiffs_from_ncs(nc_path,image_name):
    '''
    Creates GeoTIFF files from the NetCDF4 files.
    
    Input parameters:
    nc_path: filepath of the NetCDF4 file.
    image_name: Output name of the GeoTIFF
    '''
    
    tiff_file = xr.open_dataset(nc_path)
    var = tiff_file['Inundation']
    var = var.rio.set_spatial_dims('lon','lat')
    var.rio.set_crs("epsg:4326")
    var.rio.to_raster(r"tiffs/"+image_name+r".tif")
    tiff_file.close()
    

In [13]:
create_nc_from_images(23.994140625,max(lats),90,max(lons),'final.jpg','final')

'NCs/final.nc'

In [16]:
create_tiffs_from_ncs('NCs/final.nc','final')